# CS4243 Computer Vision Project


## Part 1: Setting up


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
# Function estimation using neural network, libraries

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from numpy import asarray
from matplotlib import pyplot as plt
import math as m
import random as r
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Part 2: Train our model
We first load data using the `tf.keras.utils.image_dataset_from_directory` utility. We split the images from the train directory into two, to be used for training of the model and validating its performance (note that it is not for testing the final performance of the model).

You can view the output from these datasets which we loaded.

In [ ]:
# setting the train and evaluation dataset directories. Change the below to your own path
# root_path = '/content/drive/MyDrive/CS4243 Project/data/frames/'
train_path = '/content/drive/MyDrive/CS4243 Project/data/image_data_cleaned'
test_path = '/content/drive/MyDrive/CS4243 Project/data/test_image_data_cleaned'


In [ ]:
image_size = (256,256)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    seed=110,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    seed=110,
    image_size=image_size,
    batch_size=batch_size,
)

Found 3363 files belonging to 2 classes.
Found 1931 files belonging to 2 classes.


#### Showing the images

In [ ]:
# showing the images
# #
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(4):
#         ax = plt.subplot(2, 2, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(int(labels[i]))
#         plt.axis("off")

### Data augmentation
Using horizontal flip, and random rotation rotation factor is between 0 to 0.1*2pi


In [ ]:
# data augmentation, using horizontal flip, and random rotation
# rotation factor is between 0 to 0.1*2pi
#
mean = [0.485, 0.456, 0.406]
var = np.power(np.array([0.229, 0.224, 0.225]), 2)

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.2),
        # layers.experimental.preprocessing.Normalization(mean=mean, variance=var),
        layers.experimental.preprocessing.Resizing(height=image_size[0], width=image_size[1]),
    ]
)


We define our model here which is essentially a Convolutional Neural Network. If you are not familiar with CNNs, I would recommend reading this <a href="https://aigents.co/data-science-blog/publication/introduction-to-convolutional-neural-networks-cnns">article</a> and this fun <a href="https://setosa.io/ev/image-kernels/">playground</a> (full credits to their corresponding authors.)

Some key points to help you understand some components below:
- Input() is used to instantiate a Keras tensor. It is more of a symbolic use rather than it meaning a mathematical operation - it's a way to define how the input data to the model should look.
- Batch normalization normalizes the activations of a layer to have zero mean and unit variance, helping to stabilize and accelerate training by reducing internal covariate shift.
- Residual refers to residual connections which allow gradients to "skip" layers by adding the original input to the output, helping to mitigate the vanishing gradient problem and enabling deeper networks to be trained more effectively.

FYI: Prof Amir uses Separable Convolutions layers here instead of the normal Conv2D layers you might be used to. Just note that this is just a variant of Convolution Layers, and at your own time may read up more about it. One resource is this <a href="https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728">link</a>. Understanding this specific layer is not the objective of today's lab.

In [ ]:
def make_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual
    for size in [128, 256, 512, 728]:

        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    activation = "sigmoid"
    units = 1

    x = layers.Dropout(0.5)(x)
    x = layers.Dense(25, activation='relu')(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)
# from tensorflow import keras
# from tensorflow.keras import layers

# def make_model(input_shape):
#     inputs = keras.Input(shape=input_shape)
#     x = data_augmentation(inputs)
#     x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
#     x = layers.Conv2D(16, 3, strides=2, padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)

#     x = layers.Conv2D(32, 3, padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)

#     for size in [64, 128, 256]:
#         x = layers.SeparableConv2D(size, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)
#         x = layers.Activation("relu")(x)
#         x = layers.Dropout(0.5)(x)
#         x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

#     x = layers.SeparableConv2D(512, 3, padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)

#     x = layers.GlobalAveragePooling2D()(x)

#     # Add dropout layers
#     x = layers.Dropout(0.5)(x)

#     x = layers.Dense(128, activation='relu')(x)

#     # Add another dropout layer
#     x = layers.Dropout(0.5)(x)

#     outputs = layers.Dense(1, activation='sigmoid')(x)

#     return keras.Model(inputs, outputs)

# def make_model(input_shape): # JEVNET
#     inputs = keras.Input(shape=input_shape)
#     x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)

#     # Convolutional Block 1
#     x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
#     x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
#     x = layers.MaxPooling2D(pool_size=(2, 2))(x)
#     x = layers.BatchNormalization()(x)

#     # Convolutional Block 2
#     x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
#     x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
#     x = layers.MaxPooling2D(pool_size=(2, 2))(x)
#     x = layers.BatchNormalization()(x)

#     # # Convolutional Block 3
#     # x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
#     # x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
#     # x = layers.MaxPooling2D(pool_size=(2, 2))(x)
#     # x = layers.BatchNormalization()(x)

#     x = layers.GlobalAveragePooling2D()(x)

#     x = layers.Dense(256, activation='relu')(x)
#     x = layers.Dropout(0.5)(x)  # Adding dropout for regularization
#     x = layers.Dense(128, activation='relu')(x)
#     x = layers.Dropout(0.3)(x)  # Adding dropout for regularization

#     outputs = layers.Dense(1, activation='sigmoid')(x)

#     model = keras.Model(inputs, outputs)

#     # # Compile the model
#     # model.compile(optimizer='adam',
#     #               loss='binary_crossentropy',
#     #               metrics=['accuracy'])

#     return model


As we have learnt from week 10 lab session, we define our model, `compile` to configure our ANN's learning process, and use the `fit` method to start the training process of our model.

In [ ]:
model = make_model(input_shape=image_size + (3,) )

In [ ]:
# compiling and training our model

epochs = 4
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
# This is a handy function in Keras that lets you look at your model which you have just compiled.
# Personally, looking at the output shape is particularly useful when you do CV

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 256, 256, 3)          0         ['input_1[0][0]']             
                                                                                                  
 rescaling (Rescaling)       (None, 256, 256, 3)          0         ['sequential[0][0]']          
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         896       ['rescaling[0][0]']           
                                                                                              

In [ ]:
history = model.fit(
    train_ds, epochs=epochs, validation_data=val_ds,
)

Epoch 1/4
  9/106 [=>............................] - ETA: 29:15 - loss: 0.7034 - accuracy: 0.6181

KeyboardInterrupt: ignored

#### Accuracy Curve

In [ ]:
# Accuracy curve
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Loss Curve

In [ ]:
# Loss curve
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#### Testing classifier with images

In [ ]:
# need a way to check the images and quantify them